# SmartCare Analytics 

In [ ]:
"""
Système d'extraction et d'analyse des données hospitalières
Hôpitaux Universitaires Pitié Salpêtrière - Charles Foix
"""

import pdfplumber
import pandas as pd
import re
from pathlib import Path
import json
from typing import Dict, List, Any


class HospitalDataExtractor:
    """Classe pour extraire et structurer les données du rapport hospitalier"""
    
    def __init__(self, pdf_path: str):
        self.pdf_path = pdf_path
        self.data = {
            'informations_generales': {},
            'activites_hospitalieres': {},
            'ressources_humaines': {},
            'equipements': {},
            'finances': {},
            'qualite': {},
            'recherche': {},
            'logistique': {},
            'pathologies': {}
        }
    
    def extract_text_from_pdf(self) -> str:
        """Extrait tout le texte du PDF"""
        full_text = ""
        try:
            with pdfplumber.open(self.pdf_path) as pdf:
                for page in pdf.pages:
                    text = page.extract_text()
                    if text:
                        full_text += text + "\n"
        except Exception as e:
            print(f"Erreur lors de l'extraction du texte: {e}")
        return full_text
    
    def extract_general_info(self, text: str):
        """Extrait les informations générales de l'hôpital"""
        general_info = {}
        
        # Extraction des grands chiffres clés
        patterns = {
            'nombre_lits': r'Nombre de lits\s+(\d+(?:\s+\d+)?)',
            'nombre_poles': r'Nombre de pôles\s+(\d+)',
            'centres_maladies_rares': r'Nombre de centres maladies rares\s+(\d+)',
            'hospitalisations_total': r'Nombre total d\'hospitalisations\s+([\d\s]+)',
            'sejours_plus_24h': r'Nombre de séjours \+ 24 h\s+([\d\s]+)',
            'sejours_ambulatoires': r'Nombre de séjours ambulatoires\s+([\d\s]+)',
            'consultations': r'Nombre de consultations\s+([\d\s]+)',
            'urgences': r'Passages aux urgences\s+([\d\s]+)',
            'naissances': r'Nombre de naissances\s+([\d\s]+)',
            'greffes': r'Nombre de greffes\s+(\d+)',
            'effectif_medecins': r'Effectif des médecins\s+([\d\s]+)',
            'effectif_paramedical': r'Effectif du personnel paramédical\s+([\d\s]+)',
            'depenses': r'Dépenses\s+([\d\s]+)\s*M€',
            'recettes': r'Recettes\s+([\d\s]+)\s*M€'
        }
        
        for key, pattern in patterns.items():
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                value = match.group(1).replace(' ', '').replace('\xa0', '')
                general_info[key] = int(value) if value.isdigit() else value
        
        # Âge moyen des patients
        age_patterns = {
            'age_moyen_psl': r'PSL\s+([\d\.]+)\s+ans',
            'age_moyen_cfx': r'CFX\s+([\d\.]+)\s+ans'
        }
        
        for key, pattern in age_patterns.items():
            match = re.search(pattern, text)
            if match:
                general_info[key] = float(match.group(1))
        
        self.data['informations_generales'] = general_info
    
    def extract_hospital_activities(self, text: str):
        """Extrait les données d'activités hospitalières"""
        activities = {}
        
        # Séjours MCO
        mco_pattern = r'Nombre de séjours MCO.*?PSL\s+(\d+(?:\s+\d+)?)\s+(\d+(?:\s+\d+)?)'
        match = re.search(mco_pattern, text)
        if match:
            activities['sejours_mco_psl_2011'] = int(match.group(1).replace(' ', ''))
            activities['sejours_mco_psl_2012'] = int(match.group(2).replace(' ', ''))
        
        # Consultations externes
        consult_pattern = r'Les consultations externes.*?PSL\s+([\d\s]+)\s+([\d\s]+)'
        match = re.search(consult_pattern, text)
        if match:
            activities['consultations_2011'] = int(match.group(1).replace(' ', ''))
            activities['consultations_2012'] = int(match.group(2).replace(' ', ''))
        
        # Urgences
        urgences_pattern = r'Passages\s+Patients Admis.*?(\d+\s+\d+)\s+(\d+\s+\d+)'
        match = re.search(urgences_pattern, text)
        if match:
            activities['passages_urgences_2012'] = int(match.group(2).split()[0].replace(' ', ''))
        
        self.data['activites_hospitalieres'] = activities
    
    def extract_human_resources(self, text: str):
        """Extrait les données des ressources humaines"""
        hr_data = {}
        
        # Personnel médical
        medical_patterns = {
            'personnels_universitaires': r'Personnels Hospitalo-Universitaires\s+(\d+)\s+(\d+)\s+(\d+)',
            'praticiens_hospitaliers': r'Praticiens hospitaliers.*?contractuels\s+(\d+)\s+(\d+)\s+(\d+)',
            'internes': r'Internes, résidents.*?(\d+)\s+(\d+)\s+(\d+)'
        }
        
        for key, pattern in medical_patterns.items():
            match = re.search(pattern, text, re.DOTALL)
            if match:
                hr_data[f'{key}_psl'] = int(match.group(1))
                hr_data[f'{key}_cfx'] = int(match.group(2))
                hr_data[f'{key}_total'] = int(match.group(3))
        
        # Personnel paramédical total
        param_pattern = r'Personnel hospitalier\s+(\d+(?:\s+\d+)?)\s+(\d+)\s+(\d+(?:\s+\d+)?)'
        match = re.search(param_pattern, text)
        if match:
            hr_data['personnel_hospitalier_psl'] = int(match.group(1).replace(' ', ''))
            hr_data['personnel_hospitalier_cfx'] = int(match.group(2))
            hr_data['personnel_hospitalier_total'] = int(match.group(3).replace(' ', ''))
        
        self.data['ressources_humaines'] = hr_data
    
    def extract_equipment(self, text: str):
        """Extrait les données des équipements"""
        equipment = {}
        
        equipment_patterns = {
            'laboratoires_psl': r'Laboratoires.*?PSL\s+(\d+)',
            'blocs_operatoires': r'Blocs opératoires.*?(\d+)',
            'irm': r'IRM.*?(\d+)(?:\s+(\d+))?',
            'scanner': r'Scanner diagnostique.*?(\d+)',
            'lits_reanimation': r'Lits de réanimation.*?(\d+)',
            'lits_soins_intensifs': r'Lits de soins intensifs.*?(\d+)'
        }
        
        for key, pattern in equipment_patterns.items():
            match = re.search(pattern, text, re.DOTALL)
            if match:
                equipment[key] = int(match.group(1))
        
        self.data['equipements'] = equipment
    
    def extract_financial_data(self, text: str):
        """Extrait les données financières"""
        finances = {}
        
        # Dépenses
        depenses_pattern = r'Charges relatives au personnel\s+([\d\s]+)\s+([\d\s]+)\s+([\d\s]+)'
        match = re.search(depenses_pattern, text)
        if match:
            finances['charges_personnel_psl'] = int(match.group(1).replace(' ', ''))
            finances['charges_personnel_cfx'] = int(match.group(2).replace(' ', ''))
            finances['charges_personnel_total'] = int(match.group(3).replace(' ', ''))
        
        # Recettes
        recettes_pattern = r'Produits de l\'assurance maladie\s+([\d\s]+)\s+([\d\s]+)\s+([\d\s]+)'
        match = re.search(recettes_pattern, text)
        if match:
            finances['produits_assurance_maladie_psl'] = int(match.group(1).replace(' ', ''))
            finances['produits_assurance_maladie_cfx'] = int(match.group(2).replace(' ', ''))
            finances['produits_assurance_maladie_total'] = int(match.group(3).replace(' ', ''))
        
        # Solde
        solde_pattern = r'Solde\s+([\d\s-]+)\s+([\d\s-]+)\s+([\d\s-]+)'
        match = re.search(solde_pattern, text)
        if match:
            finances['solde_psl'] = int(match.group(1).replace(' ', ''))
            finances['solde_cfx'] = int(match.group(2).replace(' ', '').replace('-', '-'))
            finances['solde_total'] = int(match.group(3).replace(' ', ''))
        
        self.data['finances'] = finances
    
    def extract_pathologies(self, text: str):
        """Extrait les principales pathologies traitées"""
        pathologies = {}
        
        pathology_patterns = {
            'cancereuses': r'([\d\s\.]+)\s+Pathologies\s+cancéreuses',
            'neurologiques': r'([\d\s\.]+)\s+Pathologies neurologiques',
            'genito_urinaire': r'([\d\s\.]+)\s+Pathologies\s+de l\'appareil\s+génito-urinaire',
            'orthopediques': r'([\d\s\.]+)\s+Pathologies orthopédiques',
            'digestif': r'([\d\s\.]+)\s+Pathologies\s+de l\'appareil digestif',
            'cardio_vasculaires': r'([\d\s\.]+)\s+Pathologies\s+cardio-vasculaires',
            'endocriniennes': r'([\d\s\.]+)\s+Pathologies endocriniennes',
            'infectieuses': r'([\d\s\.]+)\s+Pathologies infectieuses',
            'obstetrique': r'([\d\s\.]+)\s+Obstétrique'
        }
        
        for key, pattern in pathology_patterns.items():
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                value = match.group(1).replace(' ', '').replace('.', '')
                pathologies[key] = int(value) if value.isdigit() else value
        
        self.data['pathologies'] = pathologies
    
    def extract_quality_indicators(self, text: str):
        """Extrait les indicateurs qualité"""
        quality = {}
        
        # Satisfaction patients
        satisfaction_pattern = r'Excellent\s+([\d,\.]+)'
        matches = re.findall(satisfaction_pattern, text)
        if matches:
            quality['satisfaction_soins_excellent'] = float(matches[0].replace(',', '.'))
        
        self.data['qualite'] = quality
    
    def extract_logistics(self, text: str):
        """Extrait les données logistiques"""
        logistics = {}
        
        # Restauration
        resto_pattern = r'Restauration\s+([\d\s]+)\s+([\d\s]+)\s+([\d\s]+)'
        match = re.search(resto_pattern, text)
        if match:
            logistics['repas_psl'] = int(match.group(1).replace(' ', ''))
            logistics['repas_cfx'] = int(match.group(2).replace(' ', ''))
            logistics['repas_total'] = int(match.group(3).replace(' ', ''))
        
        # Lingerie
        lingerie_pattern = r'Lingerie \(en kg\)\s+([\d\s]+)\s+([\d\s]+)\s+([\d\s]+)'
        match = re.search(lingerie_pattern, text)
        if match:
            logistics['lingerie_kg_psl'] = int(match.group(1).replace(' ', ''))
            logistics['lingerie_kg_cfx'] = int(match.group(2).replace(' ', ''))
            logistics['lingerie_kg_total'] = int(match.group(3).replace(' ', ''))
        
        # Déchets DAOM
        daom_pattern = r'DAOM\*\s+([\d,]+)\s+([\d]+)\s+([\d,]+)'
        match = re.search(daom_pattern, text)
        if match:
            logistics['dechets_daom_psl'] = float(match.group(1).replace(',', '.'))
            logistics['dechets_daom_cfx'] = float(match.group(2))
            logistics['dechets_daom_total'] = float(match.group(3).replace(',', '.'))
        
        self.data['logistique'] = logistics
    
    def extract_research(self, text: str):
        """Extrait les données de recherche"""
        research = {}
        
        # Publications
        pub_pattern = r'Publications\*\s+([\d,]+)\s+([\d,]+)\s+(\d+)\s*%'
        match = re.search(pub_pattern, text)
        if match:
            research['publications_montant'] = float(match.group(1).replace(',', '.'))
            research['publications_aphp'] = float(match.group(2).replace(',', '.'))
            research['publications_pourcentage'] = int(match.group(3))
        
        # Essais cliniques
        essai_pattern = r'Essais Cliniques\s+([\d]+)\s+([\d,]+)\s+(\d+)\s*%'
        match = re.search(essai_pattern, text)
        if match:
            research['essais_montant'] = float(match.group(1))
            research['essais_aphp'] = float(match.group(2).replace(',', '.'))
            research['essais_pourcentage'] = int(match.group(3))
        
        self.data['recherche'] = research
    
    def extract_all_data(self):
        """Extrait toutes les données du PDF"""
        print("Extraction du texte du PDF...")
        text = self.extract_text_from_pdf()
        
        print("Extraction des informations générales...")
        self.extract_general_info(text)
        
        print("Extraction des activités hospitalières...")
        self.extract_hospital_activities(text)
        
        print("Extraction des ressources humaines...")
        self.extract_human_resources(text)
        
        print("Extraction des équipements...")
        self.extract_equipment(text)
        
        print("Extraction des données financières...")
        self.extract_financial_data(text)
        
        print("Extraction des pathologies...")
        self.extract_pathologies(text)
        
        print("Extraction des indicateurs qualité...")
        self.extract_quality_indicators(text)
        
        print("Extraction de la logistique...")
        self.extract_logistics(text)
        
        print("Extraction des données de recherche...")
        self.extract_research(text)
        
        return self.data
    
    def create_dataframes(self) -> Dict[str, pd.DataFrame]:
        """Crée des DataFrames pandas pour chaque catégorie"""
        dataframes = {}
        
        for category, data_dict in self.data.items():
            if data_dict:
                df = pd.DataFrame([data_dict])
                dataframes[category] = df
        
        return dataframes
    
    def export_to_csv(self, output_dir: str = "/mnt/user-data/outputs"):
        """Exporte toutes les données en fichiers CSV"""
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
        dataframes = self.create_dataframes()
        exported_files = []
        
        for category, df in dataframes.items():
            if not df.empty:
                filename = f"{output_dir}/hospital_{category}_2012.csv"
                df.to_csv(filename, index=False, encoding='utf-8-sig')
                exported_files.append(filename)
                print(f"✓ Exporté: {category} ({len(df.columns)} colonnes)")
        
        # Créer un fichier CSV récapitulatif
        self.create_summary_csv(output_dir)
        exported_files.append(f"{output_dir}/hospital_summary_2012.csv")
        
        return exported_files
    
    def create_summary_csv(self, output_dir: str):
        """Crée un fichier CSV récapitulatif avec les indicateurs clés"""
        summary_data = []
        
        # Informations générales
        gen_info = self.data.get('informations_generales', {})
        summary_data.append({
            'Catégorie': 'Infrastructure',
            'Indicateur': 'Nombre de lits',
            'Valeur': gen_info.get('nombre_lits', 'N/A'),
            'Unité': 'lits'
        })
        summary_data.append({
            'Catégorie': 'Infrastructure',
            'Indicateur': 'Nombre de pôles',
            'Valeur': gen_info.get('nombre_poles', 'N/A'),
            'Unité': 'pôles'
        })
        summary_data.append({
            'Catégorie': 'Activité',
            'Indicateur': 'Hospitalisations totales',
            'Valeur': gen_info.get('hospitalisations_total', 'N/A'),
            'Unité': 'séjours'
        })
        summary_data.append({
            'Catégorie': 'Activité',
            'Indicateur': 'Consultations',
            'Valeur': gen_info.get('consultations', 'N/A'),
            'Unité': 'consultations'
        })
        summary_data.append({
            'Catégorie': 'Urgences',
            'Indicateur': 'Passages aux urgences',
            'Valeur': gen_info.get('urgences', 'N/A'),
            'Unité': 'passages'
        })
        summary_data.append({
            'Catégorie': 'Ressources Humaines',
            'Indicateur': 'Effectif médecins',
            'Valeur': gen_info.get('effectif_medecins', 'N/A'),
            'Unité': 'ETP'
        })
        summary_data.append({
            'Catégorie': 'Ressources Humaines',
            'Indicateur': 'Effectif paramédical',
            'Valeur': gen_info.get('effectif_paramedical', 'N/A'),
            'Unité': 'ETP'
        })
        summary_data.append({
            'Catégorie': 'Finances',
            'Indicateur': 'Dépenses totales',
            'Valeur': gen_info.get('depenses', 'N/A'),
            'Unité': 'M€'
        })
        summary_data.append({
            'Catégorie': 'Finances',
            'Indicateur': 'Recettes totales',
            'Valeur': gen_info.get('recettes', 'N/A'),
            'Unité': 'M€'
        })
        
        # Pathologies principales
        pathologies = self.data.get('pathologies', {})
        for path_type, count in pathologies.items():
            summary_data.append({
                'Catégorie': 'Pathologies',
                'Indicateur': path_type.replace('_', ' ').title(),
                'Valeur': count,
                'Unité': 'séjours'
            })
        
        df_summary = pd.DataFrame(summary_data)
        df_summary.to_csv(f"{output_dir}/hospital_summary_2012.csv", 
                          index=False, encoding='utf-8-sig')
        print(f"✓ Exporté: Résumé général")
    
    def display_summary(self):
        """Affiche un résumé des données extraites"""
        print("\n" + "="*80)
        print(" RÉSUMÉ DES DONNÉES EXTRAITES - HÔPITAUX PSL-CFX 2012")
        print("="*80 + "\n")
        
        gen_info = self.data.get('informations_generales', {})
        
        print("📊 INFORMATIONS GÉNÉRALES")
        print("-" * 80)
        print(f"  • Nombre de lits: {gen_info.get('nombre_lits', 'N/A')}")
        print(f"  • Nombre de pôles: {gen_info.get('nombre_poles', 'N/A')}")
        print(f"  • Centres maladies rares: {gen_info.get('centres_maladies_rares', 'N/A')}")
        hosp = gen_info.get('hospitalisations_total', 'N/A')
        print(f"  • Hospitalisations totales: {hosp if hosp == 'N/A' else f'{hosp:,}'.replace(',', ' ')}")
        cons = gen_info.get('consultations', 'N/A')
        print(f"  • Consultations: {cons if cons == 'N/A' else f'{cons:,}'.replace(',', ' ')}")
        urg = gen_info.get('urgences', 'N/A')
        print(f"  • Passages aux urgences: {urg if urg == 'N/A' else f'{urg:,}'.replace(',', ' ')}")
        print(f"  • Naissances: {gen_info.get('naissances', 'N/A')}")
        print(f"  • Greffes: {gen_info.get('greffes', 'N/A')}")
        
        print(f"\n👥 RESSOURCES HUMAINES")
        print("-" * 80)
        print(f"  • Médecins: {gen_info.get('effectif_medecins', 'N/A')}")
        param = gen_info.get('effectif_paramedical', 'N/A')
        print(f"  • Personnel paramédical: {param if param == 'N/A' else f'{param:,}'.replace(',', ' ')}")
        
        print(f"\n💰 FINANCES")
        print("-" * 80)
        print(f"  • Dépenses: {gen_info.get('depenses', 'N/A')} M€")
        print(f"  • Recettes: {gen_info.get('recettes', 'N/A')} M€")
        
        pathologies = self.data.get('pathologies', {})
        if pathologies:
            print(f"\n🏥 PRINCIPALES PATHOLOGIES (Top 5)")
            print("-" * 80)
            sorted_patho = sorted(pathologies.items(), 
                                key=lambda x: int(x[1]) if isinstance(x[1], (int, str)) and str(x[1]).isdigit() else 0, 
                                reverse=True)[:5]
            for i, (name, count) in enumerate(sorted_patho, 1):
                count_str = f"{count:,}".replace(',', ' ') if isinstance(count, int) else count
                print(f"  {i}. {name.replace('_', ' ').title()}: {count_str} séjours")
        
        print("\n" + "="*80)
        print(f"Total de catégories extraites: {len([v for v in self.data.values() if v])}")
        print("="*80 + "\n")


def main():
    """Fonction principale"""
    # Chemin du PDF
    pdf_path = "/mnt/user-data/uploads/SLP-CHF2012.pdf"
    
    print("="*80)
    print(" EXTRACTEUR DE DONNÉES HOSPITALIÈRES PSL-CFX 2012")
    print("="*80 + "\n")
    
    # Créer l'extracteur
    extractor = HospitalDataExtractor(pdf_path)
    
    # Extraire toutes les données
    data = extractor.extract_all_data()
    
    # Afficher le résumé
    extractor.display_summary()
    
    # Exporter en CSV
    print("📁 Exportation des données en fichiers CSV...")
    exported_files = extractor.export_to_csv()
    
    print(f"\n✅ Exportation terminée! {len(exported_files)} fichiers créés.")
    print("\nFichiers générés:")
    for file in exported_files:
        print(f"  • {Path(file).name}")
    
    # Sauvegarder aussi en JSON
    json_path = "/mnt/user-data/outputs/hospital_data_complete_2012.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"\n  • hospital_data_complete_2012.json")
    
    print("\n" + "="*80)
    print(" Traitement terminé avec succès!")
    print("="*80)
    
    return data, exported_files


if __name__ == "__main__":
    data, files = main()